In [1]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import os
from PIL import Image


ModuleNotFoundError: No module named 'torchvision'

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet norms
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_dataset = datasets.ImageFolder('data/train_butterflies/train_split', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [6]:
class ButterflyTestDataset(Dataset):
    def __init__(self, root, transform):
        self.paths = sorted(os.listdir(root))
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image_path = os.path.join(self.root, self.paths[idx])
        image = Image.open(image_path).convert('RGB')
        return self.transform(image), self.paths[idx]  # return image & filename
    
test_path = 'data/test_butterflies/valid'
test_dataset = ButterflyTestDataset(test_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [15]:
import torch.nn as nn

class ButterflyNet(nn.Module):
    def __init__(self, num_classes=50):
        super(ButterflyNet, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 224x224 -> 224x224
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),                             # 224x224 -> 112x112

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 112x112 -> 112x112
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),                             # 112x112 -> 56x56

            nn.Conv2d(64, 128, kernel_size=3, padding=1),# 56x56 -> 56x56
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),                             # 56x56 -> 28x28

            nn.Conv2d(128, 256, kernel_size=3, padding=1),# 28x28 -> 28x28
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),                              # 28x28 -> 14x14
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.classifier(x)
        return x


device = torch.device('cpu')

In [16]:
model = ButterflyNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [17]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss:.4f}, Accuracy: {acc:.4f}")


In [18]:
train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch 1/10 - Loss: 827.5553, Accuracy: 0.1199


KeyboardInterrupt: 